## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-19-17-51-39 +0000,bbc,Judge strikes down 'superfluous' Trump lawsuit...,https://www.bbc.com/news/articles/c62n7025wdgo...
1,2025-09-19-17-50-00 +0000,wsj,Judge Scraps Trump’s $15 Billion New York Time...,https://www.wsj.com/us-news/law/judge-scraps-t...
2,2025-09-19-17-50-00 +0000,wsj,Opinion | Newsom Falls for the ‘Red Moocher St...,https://www.wsj.com/opinion/newsom-falls-for-t...
3,2025-09-19-17-49-00 +0000,wsj,Opinion | Europe Tries to Export Censorship to...,https://www.wsj.com/opinion/europe-tries-to-ex...
4,2025-09-19-17-48-53 +0000,nyt,"Trump and China, in Vague Statements, Suggest ...",https://www.nytimes.com/2025/09/19/business/tr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,73
6,new,26
166,kimmel,23
297,jimmy,17
89,up,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
45,2025-09-19-16-26-39 +0000,nypost,Florida judge strikes Trump’s $15B defamation ...,https://nypost.com/2025/09/19/us-news/florida-...,154
312,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...,149
303,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,147
38,2025-09-19-16-37-03 +0000,wapo,Judge strikes down Trump’s $15 billion suit ag...,https://www.washingtonpost.com/business/2025/0...,144
8,2025-09-19-17-44-30 +0000,nyt,Judge Dismisses Trump’s $15 Billion Lawsuit Ag...,https://www.nytimes.com/2025/09/19/business/me...,142


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
45,154,2025-09-19-16-26-39 +0000,nypost,Florida judge strikes Trump’s $15B defamation ...,https://nypost.com/2025/09/19/us-news/florida-...
41,91,2025-09-19-16-31-00 +0000,wsj,Some of America’s biggest late-night hosts str...,https://www.wsj.com/business/media/jimmy-kimme...
13,79,2025-09-19-17-30-00 +0000,wsj,Senate Democrats blocked Republicans’ stopgap ...,https://www.wsj.com/politics/policy/government...
303,69,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...
312,58,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...
26,48,2025-09-19-17-05-00 +0000,wsj,Stephen Miran argues for aggressive interest-r...,https://www.wsj.com/economy/central-banking/st...
64,42,2025-09-19-15-45-33 +0000,bbc,Trump says Xi approves of TikTok deal as leade...,https://www.bbc.com/news/articles/c4g7l7yl832o...
227,37,2025-09-19-01-38-44 +0000,bbc,Canada and Mexico agree to deepen ties amid Tr...,https://www.bbc.com/news/articles/cwy8r042nx2o...
118,36,2025-09-19-13-14-00 +0000,wsj,An awkward image has emerged for Illinois Gov....,https://www.wsj.com/us-news/jb-pritzker-fighti...
322,32,2025-09-18-18-59-08 +0000,bbc,Two shot dead in attack near West Bank-Jordan ...,https://www.bbc.com/news/articles/cr5qv0qmgmjo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
